In [15]:
%matplotlib inline
import matplotlib
# matplotlib.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

import pandas as pd
import numpy as np
pd.options.display.max_columns = 10
pd.options.display.max_rows = 10

import warnings
warnings.filterwarnings('ignore')
from chatto_transform.sessions.mimic import mimic_common

In [16]:
mimic_common.sql()

Loaded SELECT cmed.subject_id, cmed.hadm_id AS icu_subject_id, icu_hadm_id, 
cmed.transfertime AS icu_transfertime,
lab.charttime AS k_charttime, lab.value AS k_value, lab.valuenum AS k_valuenum, lab.uom AS k_uom, lab.hadm_id AS k_hadm_id, lab.itemid AS k_itemid
FROM
(SELECT subject_id, hadm_id, transfertime
   FROM 
   mimiciii.services) AS cmed
LEFT JOIN mimiciii.labevents AS lab
   ON lab.subject_id = cmed.subject_id
   AND lab.itemid = 50971 and stored in loaded_sql[3]


Traceback (most recent call last):
  File "/Users/yankou/Documents/CriticalDatathon/Electric-Rhythms-master/chatto_transform/sessions/mimic/mimic_common.py", line 40, in sql_exception
    yield
  File "/Users/yankou/Documents/CriticalDatathon/Electric-Rhythms-master/chatto_transform/sessions/mimic/mimic_common.py", line 94, in load_sql
    return loader.load()
  File "/Users/yankou/Documents/CriticalDatathon/Electric-Rhythms-master/chatto_transform/datastores/datastore_base.py", line 13, in load
    result = self._load()
  File "/Users/yankou/Documents/CriticalDatathon/Electric-Rhythms-master/chatto_transform/datastores/sqlalchemy_datastore.py", line 406, in _load
    df = fast_sql_to_df(self.query, self.schema)
  File "/Users/yankou/Documents/CriticalDatathon/Electric-Rhythms-master/chatto_transform/datastores/sqlalchemy_datastore.py", line 141, in fast_sql_to_df
    return fast_postgresql_to_df(table, schema)
  File "/Users/yankou/Documents/CriticalDatathon/Electric-Rhythms-master/ch

Loaded SELECT cmed.subject_id, cmed.hadm_id AS cmed_hadm_id, cmed.transfertime AS cmed_transfertime,
   lab.charttime AS k_charttime, lab.value AS k_value, lab.valuenum AS k_valuenum, lab.uom AS k_uom, lab.hadm_id AS k_hadm_id, lab.itemid AS k_itemid
FROM
(SELECT subject_id, hadm_id, transfertime
   FROM 
   mimiciii.services) AS cmed
LEFT JOIN mimiciii.labevents AS lab
   ON lab.subject_id = cmed.subject_id
   AND lab.itemid = 50971 and stored in loaded_sql[4]


In [ ]:
df = mimic_common.loaded_sql[4]
df.to_csv('ktable_allICU.csv')

# Get blood pressure from chartevents
### SQL query:

#### 1. Get itemid from d_items for blood pressure measurements:

SELECT *
	FROM MIMICIII.D_ITEMS
	WHERE label ILIKE 'FEM ART'
	    OR label ILIKE 'FEMORAL SHEATH'
	    OR label ILIKE 'ABP Alarm'
	    OR label ILIKE 'ABP'
	    OR label ILIKE 'Arterial BP Mean'
	    OR label ILIKE 'Fem sheath MAP'
	    OR label ILIKE 'Femoral ABP'
	    OR label ILIKE 'map'
	    OR label ILIKE 'Arterial Blood Pressure mean'
	    OR label ILIKE 'Non Invasive Blood Pressure mean'
	    OR label ILIKE 'Non-Invasive Blood Pressure Alarm - Low'
	    OR label ILIKE 'art mean'
	    OR label ILIKE 'arterial mean #2'

#### 2. Get chartevents rows for blood presure, quick query (save to csv file)

In [ ]:
from sqlalchemy.sql import text
from chatto_transform.sessions.mimic import mimic_login
from chatto_transform.datastores.sqlalchemy_datastore import fast_postgresql_to_csv

In [ ]:

query = text("""
SELECT * FROM
    (SELECT * FROM mimiciii.chartevents 
    WHERE itemid=52) as C

    JOIN 
        (SELECT subject_id, hadm_id, transfertime
        FROM mimiciii.services
            WHERE curr_service = 'CMED') AS cmed
    ON C.subject_id = cmed.subject_id
""").compile()

engine = mimic_login.get_engine()
query.bind = engine

directory = mimic_login.get_local_storage_dir()
fast_postgresql_to_csv(query, directory+'/blood_pressure.csv')

### Prototyping: limit to 100,000 records in chartevents

In [ ]:
SELECT * FROM
    (SELECT * FROM mimiciii.chartevents 
    WHERE itemid=52
        LIMIT 100000) as C

    JOIN 
        (SELECT subject_id, hadm_id, transfertime
        FROM mimiciii.services
        WHERE curr_service = 'CMED') AS cmed
    ON C.subject_id = cmed.subject_id

In [14]:
## load data
bptable = pd.read_csv('ABPmean_100000.csv')
ktable = pd.read_csv('ktable.csv')
bptable

,Unnamed: 0,row_id,subject_id,hadm_id,icustay_id,...,resultstatus,stopped,subject_id.1,hadm_id.1,transfertime
0,0,71255047,154,162891,250579,...,NaN,NotStopd,154,102354,2127-12-23 18:47:00
1,1,71255029,154,162891,250579,...,NaN,NotStopd,154,102354,2127-12-23 18:47:00
2,2,71255016,154,162891,250579,...,NaN,NotStopd,154,102354,2127-12-23 18:47:00
3,3,71255003,154,162891,250579,...,NaN,NotStopd,154,102354,2127-12-23 18:47:00
4,4,71254847,154,162891,250579,...,NaN,NotStopd,154,102354,2127-12-23 18:47:00
...,...,...,...,...,...,...,...,...,...,...,...
26947,26947,78190773,1395,139826,206305,...,NaN,NotStopd,1395,139826,2155-08-13 20:40:29
26948,26948,78190746,1395,139826,206305,...,NaN,NotStopd,1395,139826,2155-08-13 20:40:29
26949,26949,78189823,1395,139826,206305,...,NaN,NotStopd,1395,139826,2155-08-13 20:40:29
26950,26950,78190470,1395,139826,206305,...,NaN,NotStopd,1395,139826,2155-08-13 20:40:29


In [12]:
bptable.head()

,Unnamed: 0,row_id,subject_id,hadm_id,icustay_id,...,resultstatus,stopped,subject_id.1,hadm_id.1,transfertime
0,0,71255047,154,162891,250579,...,NaN,NotStopd,154,102354,2127-12-23 18:47:00
1,1,71255029,154,162891,250579,...,NaN,NotStopd,154,102354,2127-12-23 18:47:00
2,2,71255016,154,162891,250579,...,NaN,NotStopd,154,102354,2127-12-23 18:47:00
3,3,71255003,154,162891,250579,...,NaN,NotStopd,154,102354,2127-12-23 18:47:00
4,4,71254847,154,162891,250579,...,NaN,NotStopd,154,102354,2127-12-23 18:47:00
